In [1]:
## MMSegmentation Framework을 사용하기 위한 Install
# Install PyTorch
!pip install torch==1.12.0 torchvision --extra-index-url https://download.pytorch.org/whl/cu113
# Install MMCV
!pip install openmim
!mim install mmcv-full==1.6.0
# git clone
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     |██████████████▌                 | 834.1 MB 1.2 MB/s eta 0:14:01tcmalloc: large alloc 1147494400 bytes == 0x3a59e000 @  0x7f4c897bc615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████▍             | 1055.7 MB 1.2 MB/s eta 0:10:40tcmalloc: large alloc 1434370048 bytes == 0x7ebf4000 @  0x7f4c897bc615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████████████▎        | 

In [2]:
!pip install -q --upgrade wandb

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 62.1 MB/s 
     |████████████████████████████████| 157 kB 96.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 88.6 MB/s 
     |████████████████████████████████| 157 kB 80.0 MB/s 
     |████████████████████████████████| 157 kB 86.8 MB/s 
     |████████████████████████████████| 156 kB 92.5 MB/s 


In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: jcjang. Use `wandb login --relogin` to force relogin


True

In [5]:
!pwd

/content/mmsegmentation


In [6]:
!mkdir kaggle_dataset

In [ ]:
!gdown 1787fZejOECLSAWpMg8fDwDgWSz6gTusT # multi class 256x256
!unzip /content/mmsegmentation/Multi_256x256_MMSeg.zip -d ./kaggle_dataset

In [8]:
!mkdir ./kaggle_dataset/splits

In [9]:
from glob import glob
import numpy as np
import cv2
import os
from sklearn.model_selection import StratifiedKFold

Fold = 10
all_mask_files = glob("/content/mmsegmentation/kaggle_dataset/masks/*")
masks = []
num_mask = np.zeros((6,Fold))

for i in range(len(all_mask_files)):
    mask = cv2.imread(all_mask_files[i])
    masks.append(mask.max())

## train_valid_split, KFold : 교차검증
split = list(StratifiedKFold(n_splits=Fold, shuffle=True, random_state=2022).split(all_mask_files, masks))
for fold, (train_idx, valid_idx) in enumerate(split):
    for i in valid_idx:
        num_mask[masks[i]]+=1
    with open(f"/content/mmsegmentation/kaggle_dataset/splits/fold_{fold}.txt", "w") as f:
        for idx in train_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")
    with open(f"/content/mmsegmentation/kaggle_dataset/splits/valid_{fold}.txt", "w") as f:
        for idx in valid_idx:
            f.write(os.path.basename(all_mask_files[idx])[:-4] + "\n")
print(num_mask)

[[947. 947. 947. 947. 947. 947. 947. 947. 947. 947.]
 [396. 396. 396. 396. 396. 396. 396. 396. 396. 396.]
 [559. 559. 559. 559. 559. 559. 559. 559. 559. 559.]
 [435. 435. 435. 435. 435. 435. 435. 435. 435. 435.]
 [282. 282. 282. 282. 282. 282. 282. 282. 282. 282.]
 [172. 172. 172. 172. 172. 172. 172. 172. 172. 172.]]


In [10]:
!mkdir /content/mmsegmentation/configs/_base_/use_configs

In [11]:
!cp /content/mmsegmentation/configs/_base_/datasets/stare.py /content/mmsegmentation/configs/_base_/use_configs/stare.py
!cp /content/mmsegmentation/configs/_base_/models/deeplabv3_unet_s5-d16.py /content/mmsegmentation/configs/_base_/use_configs/deeplabv3_unet_s5-d16.py
!cp /content/mmsegmentation/configs/_base_/schedules/schedule_20k.py /content/mmsegmentation/configs/_base_/use_configs/schedule_20k.py
!cp /content/mmsegmentation/configs/_base_/default_runtime.py /content/mmsegmentation/configs/_base_/use_configs/default_runtime.py

In [12]:
!touch /content/mmsegmentation/configs/_base_/use_configs/config.py

In [14]:
!python /content/mmsegmentation/tools/train.py /content/mmsegmentation/configs/_base_/use_configs/config.py

Traceback (most recent call last):
  File "/content/mmsegmentation/tools/train.py", line 242, in <module>
    main()
  File "/content/mmsegmentation/tools/train.py", line 167, in main
    logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 519, in __getattr__
    return getattr(self._cfg_dict, name)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 50, in __getattr__
    raise ex
AttributeError: 'ConfigDict' object has no attribute 'log_level'
